#### Instructions

General: Put your OpenAI key in the .env file or it broke<br><br>

For spon: go to the sponsorship tracker and download as CSV. Delete the rows you don't care about on the first page. Put it in the src folder and name it spon.csv.

Then run this Jupyter Notebook and it'll spit out a CSV in the output folder for your copy pasting enjoyment


##### Dependencies


In [11]:
import os
from openai import OpenAI
import dotenv
import pandas as pd

dotenv.load_dotenv()

# Set up OpenAI API credentials
# client = OpenAI()
client = OpenAI()

##### Prompts


In [12]:
template = """
Hello [Name]!

Hope you are doing well! My name is Ray and I’m a student studying Business & Computer Science at Western University. I’m an organizer for Hack Western and am thrilled to invite [company name] to the 11th iteration of Hack Western, one of Canada’s largest student-run hackathons! 

[1-2 sentences: personalization piece – mention something good you’ve heard about the company, or if you find something cool, and why they’re a good fit. If available, work in their role and previous sponsorship with Hack Western 11.]

From November 29 to December 1, 2024, over 350 of North America's brightest students will gather at Western University for 36 hours to build technological solutions that solve real-world problems. We saw 70+ projects demoed last year, ranging from metaverse technologies, AI-powered interfaces, to blockchain applications. 

It is a unique opportunity for [company] to:
Recruit top talent. Hack Western attracts bright students who come from diverse backgrounds. Western is known as a thriving entrepreneurial hub (#3 in Canada), and our hackathon hosts over 40% of students from other schools such as University of Waterloo and Toronto.
Showcase your company. From sponsored challenges, networking booths, to workshops, there are numerous opportunities for deeper, meaningful interaction between your company and students.
Inspire students. Empower the next generation of tech leaders and see firsthand how transformative hackathons can be.

I’d be happy to send you more details or set up a call, thank you!

Best,

Ray Wang
Organizer, Hack Western 11
hackwestern.com
raymond@hackwestern.com | rwang.hba2025@ivey.ca
"""

prompt = """
You are a student organizer for Hack Western, one of Canada's largest student-run hackathons. You are reaching out to a company to invite them to sponsor the event. Write an email to the company that includes the following. Ensure that your email is professional and engaging, and that you include all the necessary information based on the company and the areas denoted with [] brackets.

You will be provided with a template, as well as information about the company. This is just a suggestion; please improve it signifciantly to better fit the company and message. You must only return the email body. 
"""

##### Funding Sam Altman's Scarlett Johansson obsession


In [13]:
def gpt_4o_call(details, template):
    """
    Uses the GPT-4o model on Azure OpenAI endpoint to generate a conversational response.
    """
    messages = [
        {
            "role": "system",
            "content": prompt,
        },
        {
            "role": "user",
            "content": f"The provided information about this firm is: {details}",
        },
        {"role": "user", "content": f"The provided template is: {template}"},
    ]

    response = client.chat.completions.create(
        model="gpt-4o",  # for OpenAI
        # max_tokens=500,
        n=1,
        stop=None,
        temperature=0.5,
        messages=messages,
    )

    return response.choices[0].message.content.strip()

In [14]:
# Ingest company details as pandas dataframe; the first row is headers
df = pd.read_csv("src/spon.csv")

# Here are some sample rows from the sponsor data in CSV
# sample = """
# Company,HW11 Assignee,Status,Last HW Sponsor,Industry,Name,Role,Email,LinkedIn,Phone Number,Notes,Past Sponsor
# 1517 Fund,Raymond,Reached out,HW9,VC,Nick,,nick@1517fund.com,,,,
# IBM,Raymond,Reached out,HW9,Enterprise Software,Judith Fernando,Campus Acquisition Talent Partner,judith.fernando@ibm.com,https://www.linkedin.com/in/judith-fernando15/,,,
# Accenture,Raymond,Reached out,HW10,Consulting,,Entry Level Recruiting,zeynep.uncu@accenture.com,,,reached out to us directly,
# """

# Create an output dataframe with schema Company | Title | Email
output_df = pd.DataFrame(columns=["Company", "Title", "Email"])

# Iterate over each row in the dataframe and populate the output

# Create an empty list to store the data
data_list = []

# Iterate over each row in the dataframe and populate the data list
for index, row in df.iterrows():
    company = row["Company"]
    title = (
        "Hack Western 11 Sponsorship Invitation | Connect and Empower Top Talent | Partnership with "
        + company
    )
    email = row["Email"]
    # Details should consist of labels and values of the relevant fields from the datasheet: Company, Status, Last HW Sponsor, Industry, Name, Role, Email, LinkedIn, Phone Number, Notes, Past Sponsor
    details = f"Company: {company}, Status: {row['Status']}, Last Time They Sponsored HW: {row['Last HW Sponsor']}, Industry: {row['Industry']}, Name: {row['Name']}, Role: {row['Role']}, Email: {email}, LinkedIn: {row['LinkedIn']}, Phone Number: {row['Phone Number']}, Notes: {row['Notes']}"

    # LOLOLOL IF SOMEONE WANTS TO IMPLEMENT MULTITHREADING HERE PLS DO BC I'M TOO LAZY
    response = gpt_4o_call(details, template)

    # Append the data as a list to the data list
    data_list.append([company, title, email, response])

# Create the output dataframe using the data list
output_df = pd.DataFrame(data_list, columns=["Company", "Title", "Email", "Response"])

# Write the output dataframe to a CSV file
output_df.to_csv("output/spon_output.csv", index=False)